In [ ]:
### goja是什么

goja实现了ECMAScript 5.1(+)

* 特性

```
1. 完整支持ECMAScript 5.1
2. 测试通过了ECMAScript官方所有测试用例
3. 支持任何能在babel, typescript编译器中运行任何ES5
4. 支持sourcemap
```

### goja能够做什么

* 执行js表达式

```go
func testExpression(){
	//创建运行实例
	vm := goja.New()
	//运行表达式
	v, _ := vm.RunString("2 + 2")
	//获得结果
	num := v.Export().(int64)
	//打印结果
	fmt.Println(num)
}
```



* 执行js脚本

```go
func testScript(){
	vm := goja.New()
	registry := new(require.Registry)
	registry.Enable(vm)

	vm.RunString(`
	var console = require('console');
	var num = 2+2;
	console.log(num);
`)
}
```


* js调用go函数

```go
func Test(a int, b int) int{
	return a+b
}

func testCallGoFunc(){
	vm := goja.New()

	vm.Set("Test", Test)
	v, _ := vm.RunString(`
	Test(2, 2);
`)
	fmt.Println(v.Export().(int64))
}

```

* go调用js函数

```go
func testCallJsFunc(){
	vm := goja.New()

	//js函数
	const SCRIPT = `
function f(a, b) {
    return a+b;
}
`
	vm.RunString(SCRIPT)

	//声明函数变量
	var fn func(int, int) string
	//建立引用关系
	vm.ExportTo(vm.Get("f"), &fn)
	//执行并打印结果
	fmt.Println(fn(2, 2))
}
```

* go往js传递参数

```go
func testParamFromGoToJs(){
	type S struct {
		Field int `json:"field"`
	}

	vm := goja.New()
	vm.SetFieldNameMapper(goja.TagFieldNameMapper("json", true))
	vm.Set("s", S{Field: 42})
	res, _ := vm.RunString(`s.field`)
	fmt.Println(res.Export())
}
```

* go从js获取参数

```go
func testParamGetFromJs(){
	vm := goja.New()
	vm.RunString(`
	var num = 4;
`)

	fmt.Println(vm.Get("num").Export().(int64))
}
```

* go捕获js异常

```go
func testCatchJsException(){
	vm := goja.New()
	_, err := vm.RunString(`
	throw("js erorr");
`)
	if jserr, ok := err.(*goja.Exception); ok {
		fmt.Println(jserr)
	}
}
```

* js捕获go异常

```
var jsvm *goja.Runtime

func TestError(){
	panic(jsvm.ToValue("go Error"))
}

func testCatchGoException(){
	jsvm = goja.New()
	jsvm.Set("TestError", TestError)

	_, err := jsvm.RunString(`
try {
    TestError();
} catch(e) {
    if (e !== "Error") {
        throw e;
    }
}
`)
	if jserr, ok := err.(*goja.Exception); ok {
		fmt.Println(jserr)
	}
}
```